# Choose Number of Topics using LDA algorithm

In [9]:
import gensim
import nltk
nltk.download('wordnet')

import pandas as pd
from sklearn.manifold import TSNE
import time

from tqdm import tqdm
import numpy as np
from sklearn.manifold import TSNE

import matplotlib.pyplot as plt
%matplotlib inline

from wordcloud import WordCloud

from bokeh.plotting import figure, show, output_notebook, save
from bokeh.models import HoverTool, value, LabelSet, Legend, ColumnDataSource
output_notebook()


### BEGIN settings of pyLDAvis ### 
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
### END settings of pyLDAvis ###

[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Loading BokehJS ...

# Preprocess Documents

In [2]:
def read_json_docs(filename):
    dataset_df = pd.read_csv(filename, error_bad_lines=False)
    
    dataset = dataset_df['text']
    
    return dataset


def preprocess(raw_docs):
    """
    Preprocess documents including normalization, lemmatization, tokenization, removing stopwords.

    Args:
        raw_docs (pandas.Series)
    
    Returns:
        docs (list(list(str)))
    """
    lemmatizer = nltk.stem.WordNetLemmatizer()
    
    docs = []
    
    for d in raw_docs:
        tokens = gensim.utils.simple_preprocess(d)
        stopwords_custom = ['https', 'rt', 'amp'] 
        doc = []
        
        for t in tokens:
            if t not in gensim.parsing.preprocessing.STOPWORDS:
                if t not in stopwords_custom:
                    t = lemmatizer.lemmatize(t, pos='v') # convert past tense to present tense.
                    doc.append(t)
                    
        docs.append(doc)
        
    return docs
        
       
def create_dictionary(docs):
    """
    Create dictionary
    
    Args:
        docs (list(list(str)))
        
    Returns:
        dictionary (gensim.corpora.Dictionary)
    """
    
    return gensim.corpora.Dictionary(docs)
    
    
def get_bow_corpus(docs, dictionary):
    """
    Convert texts to encoded bows using dictionary
    
    Args:
        docs (list(list(str))):
        dictionary (gensim.corpora.Dictionary)
        
    Returns:
        bows (list(list(tuple(token_id, token_count))))
    """
    bows = [dictionary.doc2bow(d) for d in docs]
    return bows


def get_tfidf_corpus(bows):
    tfidf_model = gensim.models.TfidfModel(bows)
    corpus_tfidf = tfidf_model[bows]
    
    return corpus_tfidf, tfidf_model

# Traning Program

In [22]:
filename_dataset = './twitter_trump_2019_05.csv'
no_below = 2
no_above = 1.0
num_topics = 3
passes = 20

In [3]:
# Read file
raw_docs = read_json_docs(filename_dataset)

# Preprocess documents including normalization, lemmatization, tokenization, removing stopwords.
docs = preprocess(raw_docs)

# Create dictionary
dictionary = create_dictionary(docs)
print('Size of unfiltered dictionary: {}'.format(len(dictionary)))

# Remove common/rare words
dictionary.filter_extremes(no_below=no_below, no_above=no_above)
print('Size of filtered dictionary: {}'.format(len(dictionary)))

# Transform docs to BOWs corpus
corpus_bows = get_bow_corpus(docs, dictionary)

# Transform BOWS to TF-IDF corpus
corpus_tfidf, tfidf_model = get_tfidf_corpus(corpus_bows)


Size of unfiltered dictionary: 2724
Size of filtered dictionary: 1162


# Train LDA

In [23]:
# Train the corpus_bows
lda_model_bows = gensim.models.LdaModel(corpus_bows, num_topics=num_topics, id2word=dictionary,passes=passes)

# Train the corpus_bows
# lda_model_tfidf = gensim.models.LdaModel(corpus_tfidf,
#                                               num_topics=num_topics,
#                                               id2word=dictionary,
#                                               passes=passes)

# Choose Number of Topics using pyLDAvis

In [24]:
pyLDAvis.gensim.prepare(lda_model_bows, corpus_bows, dictionary)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.092408  0.112439       1        1  38.281746
2     -0.161437  0.011406       2        1  36.071880
0      0.069029 -0.123844       3        1  25.646383, topic_info=     Category        Freq             Term       Total  loglift  logprob
336   Default   84.000000            china   84.000000  30.0000  30.0000
996   Default   53.000000         dbongino   53.000000  29.0000  29.0000
730   Default   42.000000           tariff   42.000000  28.0000  28.0000
6     Default   45.000000          mueller   45.000000  27.0000  27.0000
27    Default   68.000000            trump   68.000000  26.0000  26.0000
531   Default   31.000000          billion   31.000000  25.0000  25.0000
68    Default   36.000000              job   36.000000  24.0000  24.0000
337   Default   28.000000             deal   28.000000  23.0000  23.0000
364   Default   20.000000           border   20.000000  22.0000  22.0000
103   Default   31.000000           report   31.000000  21.0000  21.0000
487   Default   23.000000        tomfitton   23.000000  20.0000  20.0000
666   Default   31.000000          dollars   31.000000  19.0000  19.0000
304   Default   27.000000            unite   27.000000  18.0000  18.0000
723   Default   23.000000              buy   23.000000  17.0000  17.0000
784   Default   15.000000    judicialwatch   15.000000  16.0000  16.0000
285   Default   30.000000        collusion   30.000000  15.0000  15.0000
104   Default   24.000000            thank   24.000000  14.0000  14.0000
133   Default   21.000000              usa   21.000000  13.0000  13.0000
610   Default   14.000000            build   14.000000  12.0000  12.0000
64    Default  102.000000            great  102.000000  11.0000  11.0000
48    Default   21.000000      obstruction   21.000000  10.0000  10.0000
85    Default   43.000000        democrats   43.000000   9.0000   9.0000
635   Default   16.000000         products   16.000000   8.0000   8.0000
127   Default   52.000000            state   52.000000   7.0000   7.0000
126   Default   11.000000      republicans   11.000000   6.0000   6.0000
299   Default   15.000000            trade   15.000000   5.0000   5.0000
5     Default   22.000000              fbi   22.000000   4.0000   4.0000
87    Default   70.000000        president   70.000000   3.0000   3.0000
1111  Default   15.000000            union   15.000000   2.0000   2.0000
366   Default   16.000000            obama   16.000000   1.0000   1.0000
...       ...         ...              ...         ...      ...      ...
389    Topic3    5.076192     intelligence    5.924035   1.2063  -5.9332
526    Topic3    6.354300             lead    7.620769   1.1790  -5.7086
68     Topic3   27.414886              job   36.587082   1.0722  -4.2467
764    Topic3    6.934464             city    8.483631   1.1591  -5.6212
104    Topic3   17.807375            thank   24.629835   1.0364  -4.6781
366    Topic3   12.036832            obama   16.138987   1.0675  -5.0698
95     Topic3    9.474364              spy   12.732904   1.0652  -5.3092
21     Topic3    8.881911         campaign   11.883086   1.0697  -5.3737
680    Topic3    6.677295            break    8.493144   1.1202  -5.6590
772    Topic3    7.070757            email    9.357613   1.0805  -5.6018
64     Topic3   40.765827            great  102.588791   0.4379  -3.8499
5      Topic3   13.218604              fbi   22.165813   0.8438  -4.9761
343    Topic3    9.683638             maga   14.519244   0.9557  -5.2873
25     Topic3    9.202335          foxnews   13.657146   0.9660  -5.3383
132    Topic3   26.300276  realdonaldtrump   60.175453   0.5331  -4.2882
27     Topic3   27.690058            trump   68.594582   0.4536  -4.2367
240    Topic3   16.848171             work   37.758793   0.5538  -4.7335
208    Topic3   14.798894              new   31.830978   0.5949  -4.8632
168    Topic3    7.704217           r